# get data

In [1]:
import requests
import json
import pandas as pd
import urllib.parse

In [2]:
api_key = "AE1bK67q.rabU8zbd7HiTjDZWOpW2b4VlbHfBhf2c"

In [3]:
def fetch_page(query_url, page, api_key):
    options = {"page": page, "per_page": 100, "sort": [{"patent_year": "asc"}]}
    url = query_url + '&f=' + json.dumps(fields) + '&o=' + json.dumps(options)
    response = requests.get(url, headers={"X-Api-Key": api_key})
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Your existing code to build the query
query = {
    "_and": [
        {"_gte": {"patent_date": "2020-01-01"}},
        {"_lte": {"patent_date": "2024-12-31"}},
        {
            "_or": [
                {"_text_phrase": {"patent_title": "machine learning"}},
                {"_text_phrase": {"patent_title": "deep learning"}},
                {"_text_phrase": {"patent_title": "artificial intelligence"}},
                {"_text_phrase": {"patent_title": "neural network"}},
                {"_text_phrase": {"patent_abstract": "machine learning"}},
                {"_text_phrase": {"patent_abstract": "deep learning"}},
                {"_text_phrase": {"patent_abstract": "artificial intelligence"}},
                {"_text_phrase": {"patent_abstract": "neural network"}}
            ]
        }
    ]
}


fields = ["patent_number", "patent_year", "patent_title","patent_abstract",
          "assignee_country","assignee_organization",
          "patent_num_claims","cited_patent_number","citedby_patent_number",
          "ipc_section", "ipc_class","ipc_subclass","ipc_main_group", "ipc_subgroup"]

query_string = json.dumps(query)
encoded_query = urllib.parse.quote(query_string)

base_url = 'https://api.patentsview.org/patents/query?q='
query_url = base_url + encoded_query
# Initialize an empty list to store DataFrames
dataframes = []

# Initialize a variable for the current page
current_page = 1

# Loop to fetch each page
while True:
    data = fetch_page(query_url, current_page, api_key)
    if data and data.get('patents'):
        # Convert the data to a DataFrame
        df = pd.DataFrame(data.get('patents'))
        dataframes.append(df)
        
        # Increment the page number for the next request
        current_page += 1
    else:
        # Exit the loop if there are no more results
        break

# Concatenate all DataFrames into a single DataFrame
all_patents = pd.concat(dataframes, ignore_index=True)

# Save the complete DataFrame to a CSV file
csv_file_path = 'AI_patent_data.csv'
all_patents.to_csv(csv_file_path, index=False, encoding='utf-8')
print(f"Data successfully saved to {csv_file_path}")

Data successfully saved to AI_patent_data.csv


# preprocess

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast

In [66]:
df=pd.read_csv('AI_patent_raw.csv', encoding='utf-8')

In [67]:
df.isnull().sum()

patent_number        17
patent_year          17
patent_title         17
patent_abstract      17
patent_num_claims    17
assignees            17
IPCs                 17
cited_patents        17
dtype: int64

In [68]:
df.dropna(axis=0, inplace=True)

In [69]:
df.isnull().sum()

patent_number        0
patent_year          0
patent_title         0
patent_abstract      0
patent_num_claims    0
assignees            0
IPCs                 0
cited_patents        0
dtype: int64

In [70]:
# Before code
df['assignee_country'] = None
df['assignee_organization'] = None

# 'assignees' 열을 순회하면서 값을 추출하여 열에 추가
for index, row in df.iterrows():
    row_data = ast.literal_eval(row['assignees'])
    
    # 데이터 추출
    assignee_countries = []
    assignee_organizations = []
    for item in row_data:
        assignee_country = item.get('assignee_country')
        assignee_organization = item.get('assignee_organization')
        
        # Append values to lists
        assignee_countries.append(assignee_country)
        assignee_organizations.append(assignee_organization)
        

    # 열에 추가
    df.at[index, 'assignee_country'] = ', '.join(filter(None, assignee_countries))
    df.at[index, 'assignee_organization'] = ', '.join(filter(None, assignee_organizations))


In [71]:
df

,patent_number,patent_year,patent_title,patent_abstract,patent_num_claims,assignees,IPCs,cited_patents,assignee_country,assignee_organization
0,10524461.0,2020.0,Pest detector to identify a type of pest using...,"In some implementations, a detector may includ...",20.0,"[{'assignee_country': None, 'assignee_organiza...","[{'ipc_section': 'A', 'ipc_class': '1', 'ipc_s...","[{'cited_patent_number': '6445301'}, {'cited_p...",,
1,10524737.0,2020.0,Condition detection in a virtual reality syste...,Techniques that facilitate condition detection...,17.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'A', 'ipc_class': '61', 'ipc_...","[{'cited_patent_number': '6201476'}, {'cited_p...",US,International Business Machines Corporation
2,10524761.0,2020.0,Enhanced stethoscope devices and methods,An enhanced stethoscope device and method for ...,33.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'A', 'ipc_class': '61', 'ipc_...","[{'cited_patent_number': '4413629'}, {'cited_p...",US,California Institute of Technology
3,10525563.0,2020.0,Abnormality-detecting device and method for to...,An abnormality-detecting device for detecting ...,14.0,"[{'assignee_country': 'JP', 'assignee_organiza...","[{'ipc_section': 'B', 'ipc_class': '23', 'ipc_...","[{'cited_patent_number': '5579232'}, {'cited_p...",JP,"MITSUBISHI HEAVY INDUSTRIES MACHINE TOOL CO., ..."
4,10525615.0,2020.0,Water regulated temperature controlled resin d...,Apparatus and method for drying thermoplastic ...,3.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'B', 'ipc_class': '29', 'ipc_...","[{'cited_patent_number': '3233416'}, {'cited_p...",US,"Novatec, Inc."
...,...,...,...,...,...,...,...,...,...,...
37432,12185129.0,2024.0,Utilizing invariant user behavior data for tra...,A device may receive mobile radio data identif...,20.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'H', 'ipc_class': '4', 'ipc_s...",[{'cited_patent_number': None}],US,VIAVI Solutions Inc.
37433,12185147.0,2024.0,Beam inference for multiple transmit receive p...,Various aspects of the present disclosure gene...,30.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'H', 'ipc_class': '4', 'ipc_s...",[{'cited_patent_number': '11375527'}],US,QUALCOMM Incorporated
37434,12185206.0,2024.0,Service discovery and session establishment fo...,Aspects of dynamic and interactive machine lea...,30.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'H', 'ipc_class': '4', 'ipc_s...",[{'cited_patent_number': '11974171'}],US,QUALCOMM Incorporated
37435,12185210.0,2024.0,Systems and methods for pairing devices using ...,The present disclosure is directed at pairing ...,20.0,"[{'assignee_country': 'US', 'assignee_organiza...","[{'ipc_section': 'H', 'ipc_class': '4', 'ipc_s...","[{'cited_patent_number': '8706162'}, {'cited_p...",US,Intel Corporation


In [72]:
if 'IPCs' in df.columns:
    # Preprocess IPCs column
    for index, row in df.iterrows():
        row_data = ast.literal_eval(row['IPCs'])
        
        formatted_IPCs = []
        for ipc_data in row_data:
            ipc_class = ipc_data['ipc_class']
            # Ensure ipc_class is a two-digit number
            ipc_class = ipc_class.zfill(2)
            
            ipc_section = ipc_data['ipc_section']
            ipc_subclass = ipc_data['ipc_subclass']
            ipc_main_group = ipc_data['ipc_main_group']
            # Ensure ipc_main_group is a three-digit number
            ipc_main_group = ipc_main_group.zfill(3)
            
            # Format IPC string
            formatted_IPC = f"{ipc_section}{ipc_class}{ipc_subclass}{ipc_main_group}"
            formatted_IPCs.append(formatted_IPC)
        
        # Join formatted IPCs and assign back to the IPCs column
        df.at[index, 'IPCs'] = ', '.join(formatted_IPCs)
else:
    print("Column 'IPCs' not found in the DataFrame.")

In [73]:
import ast

# 새로운 컬럼 생성
df['cited_patent_numbers'] = None

for index, row in df.iterrows():
    try:
        # 문자열을 리스트로 변환
        cited_data = ast.literal_eval(row['cited_patents'])
    except (ValueError, SyntaxError):
        cited_data = []
    
    # 값 추출 (예: 'cited_patent_number'라는 키가 있다고 가정)
    cited_numbers = []
    for item in cited_data:
        cited_number = item.get('cited_patent_number')
        if cited_number:
            cited_numbers.append(cited_number)
    
    # 콤마로 연결해 저장
    df.at[index, 'cited_patent_numbers'] = ', '.join(cited_numbers)

In [74]:
df

,patent_number,patent_year,patent_title,patent_abstract,patent_num_claims,assignees,IPCs,cited_patents,assignee_country,assignee_organization,cited_patent_numbers
0,10524461.0,2020.0,Pest detector to identify a type of pest using...,"In some implementations, a detector may includ...",20.0,"[{'assignee_country': None, 'assignee_organiza...","A01M001, G06K009, G06F009, G01S013, G06F001, G...","[{'cited_patent_number': '6445301'}, {'cited_p...",,,"6445301, 7441368, 8400348, 9385953, 9792557, 1..."
1,10524737.0,2020.0,Condition detection in a virtual reality syste...,Techniques that facilitate condition detection...,17.0,"[{'assignee_country': 'US', 'assignee_organiza...","A61B005, G06F003, A61B005, A61B005, A61B005, G...","[{'cited_patent_number': '6201476'}, {'cited_p...",US,International Business Machines Corporation,"6201476, 6433690, 8206325, 9753285"
2,10524761.0,2020.0,Enhanced stethoscope devices and methods,An enhanced stethoscope device and method for ...,33.0,"[{'assignee_country': 'US', 'assignee_organiza...","A61B007, A61B005, A61B008, A61B005, G01S015, A...","[{'cited_patent_number': '4413629'}, {'cited_p...",US,California Institute of Technology,"4413629, 4768174, 4972841, 5394880, 5798489, 5..."
3,10525563.0,2020.0,Abnormality-detecting device and method for to...,An abnormality-detecting device for detecting ...,14.0,"[{'assignee_country': 'JP', 'assignee_organiza...","B23Q017, G05B013, G05B023, B23Q017, G05B013, G...","[{'cited_patent_number': '5579232'}, {'cited_p...",JP,"MITSUBISHI HEAVY INDUSTRIES MACHINE TOOL CO., ...","5579232, 9349099, 10379517"
4,10525615.0,2020.0,Water regulated temperature controlled resin d...,Apparatus and method for drying thermoplastic ...,3.0,"[{'assignee_country': 'US', 'assignee_organiza...","B29B013, F26B021, F26B021, F26B023, F26B003, F...","[{'cited_patent_number': '3233416'}, {'cited_p...",US,"Novatec, Inc.","3233416, 4601114, 4670203, 7316079, 7568905, 1..."
...,...,...,...,...,...,...,...,...,...,...,...
37432,12185129.0,2024.0,Utilizing invariant user behavior data for tra...,A device may receive mobile radio data identif...,20.0,"[{'assignee_country': 'US', 'assignee_organiza...","H04W024, G06N005, H04W016, H04W024",[{'cited_patent_number': None}],US,VIAVI Solutions Inc.,
37433,12185147.0,2024.0,Beam inference for multiple transmit receive p...,Various aspects of the present disclosure gene...,30.0,"[{'assignee_country': 'US', 'assignee_organiza...","H04W024, H04B007, H04L005, H04W064, G06N003",[{'cited_patent_number': '11375527'}],US,QUALCOMM Incorporated,11375527
37434,12185206.0,2024.0,Service discovery and session establishment fo...,Aspects of dynamic and interactive machine lea...,30.0,"[{'assignee_country': 'US', 'assignee_organiza...","H04W004, G06N005, H04W048, H04W076, H04W080, H...",[{'cited_patent_number': '11974171'}],US,QUALCOMM Incorporated,11974171
37435,12185210.0,2024.0,Systems and methods for pairing devices using ...,The present disclosure is directed at pairing ...,20.0,"[{'assignee_country': 'US', 'assignee_organiza...","H04W004, G06F003, G06F013, G06T007, H04B007, H...","[{'cited_patent_number': '8706162'}, {'cited_p...",US,Intel Corporation,"8706162, 9436805, 10820360"


In [75]:
df.to_csv('AI_patent_processed.csv', encoding='utf-8-sig')